# Instructions

To run this experiment, first run the Setup chunk, then run the RFF section.

In [1]:
# IPython magics
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Standard library
import os
import time
import math
import gc
import statistics
import urllib.request

# Third-party libraries
import numpy as np
import pandas as pd
import torch
import gpytorch
import pynvml
import psutil
from tqdm import tqdm, trange
from matplotlib import pyplot as plt
from scipy.io import loadmat

# GPyTorch components
from gpytorch.models import ApproximateGP
from gpytorch.variational import (
    NNVariationalStrategy,
    CholeskyVariationalDistribution,
    VariationalStrategy,
)
from gpytorch.models.deep_gps import DeepGPLayer, DeepGP
from gpytorch.mlls import DeepApproximateMLL
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

# PyTorch data utilities
from torch.utils.data import TensorDataset, DataLoader

# Global settings
gpu = torch.cuda.is_available()
n_replicates = 10
smoke_test = ('CI' in os.environ)
max_vram = 0
max_ram = 0

def log_memory():
    """Log GPU and system memory usage."""
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    meminfo = pynvml.nvmlDeviceGetMemoryInfo(handle)

    max_allocated = torch.cuda.max_memory_allocated() / 1024**2  # MB
    max_reserved  = torch.cuda.max_memory_reserved()  / 1024**2  # MB
    gpu_used      = meminfo.used                   / 1024**2  # MB
    sys_used      = psutil.virtual_memory().used   / 1024**3  # GB

    print(f"[PyTorch]   Max Allocated: {max_allocated:.2f} MB | Max Reserved: {max_reserved:.2f} MB")
    print(f"[GPU VRAM]   Used (nvidia-smi): {gpu_used:.2f} MB | [System RAM]: {sys_used:.2f} GB")

    return max_allocated, max_reserved, gpu_used, sys_used

def get_mem():
    """Return current process RSS memory in MB."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024**2)

def vram_usage():
    """Track peak CUDA memory allocated."""
    global max_vram
    allocated = torch.cuda.memory_allocated()
    max_vram = max(max_vram, allocated)

def splitter(x_cpu, y_cpu, n_train=80000, n_test=20000, random_state=42, move_to_gpu=True):
    """
    Split (x_cpu, y_cpu) into train/test sets.
    Returns tensors (train_x, train_y, test_x, test_y),
    optionally moved to GPU.
    """
    assert x_cpu.shape[0] == y_cpu.shape[0], "Mismatch in number of samples"
    total = x_cpu.shape[0]
    assert n_train + n_test <= total, "Not enough samples to split"

    rng = np.random.default_rng(seed=random_state)
    idx = rng.permutation(total)

    train_idx = idx[:n_train]
    test_idx  = idx[n_train : n_train + n_test]

    train_x = x_cpu[train_idx].contiguous()
    train_y = y_cpu[train_idx].contiguous()
    test_x  = x_cpu[test_idx].contiguous()
    test_y  = y_cpu[test_idx].contiguous()

    if move_to_gpu and torch.cuda.is_available():
        train_x, train_y = train_x.cuda(), train_y.cuda()
        test_x,  test_y  = test_x.cuda(),  test_y.cuda()

    return train_x, train_y, test_x, test_y

# Load data
coords_df = pd.read_csv('Data/coordinates.csv')
expr_df   = pd.read_csv('Data/Mbp.csv')

all_x = torch.tensor(coords_df.values, dtype=torch.float32).contiguous()
all_y = torch.tensor(expr_df.iloc[:, 0].values, dtype=torch.float32).contiguous()

print("all_x shape:", all_x.shape)
print("all_y shape:", all_y.shape)

# Split into train/test
train_x, train_y, test_x, test_y = splitter(all_x, all_y, n_train=80000, n_test=20000)
print("train_x shape:", train_x.shape)
print("train_y shape:", train_y.shape)
print("test_x  shape:", test_x.shape)
print("test_y  shape:", test_y.shape)


FileNotFoundError: [Errno 2] No such file or directory: 'Data/coordinates.csv'

# RFF

In [ ]:
import tqdm, time, gc
import torch, gpytorch
from memory_profiler import memory_usage

class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RFFKernel(
                num_samples=200,  # Adjust this line to choose the number of Fourier Features
                num_dims=2
            )
        )

    def forward(self, x):
        return gpytorch.distributions.MultivariateNormal(
            self.mean_module(x),
            self.covar_module(x)
        )

n_replicates = 11
training_iterations = 64
n_train, n_test = 40_000, 20_000
random_state = 42
mse_l_rff, time_l_rff = [], []

for rep in range(n_replicates):
    print(f"\n=== Replicate {rep + 1}/{n_replicates} ===")
    train_x, train_y, test_x, test_y = splitter(all_x, all_y, n_train=40000, n_test=20000, random_state=rep)
    ram_before = get_mem() / (1024**2)
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats()
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPRegressionModel(train_x, train_y, likelihood)
    model.train(); likelihood.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.25)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    if torch.cuda.is_available():
        mll = mll.cuda()

    def train_fn():
        progress = tqdm.trange(training_iterations, desc=f"Training (rep {rep+1})", leave=False)
        for _ in progress:
            optimizer.zero_grad()
            output = model(train_x)
            loss = -mll(output, train_y)
            loss.backward()
            optimizer.step()
            progress.set_postfix(loss=loss.item())
        return None

    start_time = time.time()
    peak_ram = memory_usage(
        (train_fn,),
        max_usage=True,
        retval=False,
        interval=0.01
    )
    elapsed = time.time() - start_time
    vram_peak = torch.cuda.max_memory_allocated() / (1024**2) if torch.cuda.is_available() else None
    ram_delta = peak_ram - ram_before
    model.eval(); likelihood.eval()
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        pred = likelihood(model(test_x)).mean.cpu()
    mse = torch.mean((pred - test_y.cpu()) ** 2).item()
    mse_l_rff.append(mse)
    time_l_rff.append(elapsed)
    print(
        f"Rep {rep+1}: MSE={mse:.4f}, Time={elapsed:.2f}s, "
        f"RAM before={ram_before:.1f}MB, peak={peak_ram:.1f}MB (Δ={ram_delta:.1f}MB)"
        + (f", VRAM peak={vram_peak:.1f}MB" if vram_peak is not None else "")
    )
    del model, likelihood
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()



=== Replicate 1/11 ===


Rep 1: MSE=0.7371, Time=10.75s, RAM before=0.0MB, peak=798.2MB (Δ=798.2MB), VRAM peak=12689.6MB

=== Replicate 2/11 ===


Rep 2: MSE=0.7257, Time=10.76s, RAM before=0.0MB, peak=948.7MB (Δ=948.7MB), VRAM peak=12814.9MB

=== Replicate 3/11 ===


Rep 3: MSE=0.7316, Time=10.61s, RAM before=0.0MB, peak=948.7MB (Δ=948.7MB), VRAM peak=12814.9MB

=== Replicate 4/11 ===


Rep 4: MSE=0.7406, Time=10.61s, RAM before=0.0MB, peak=948.7MB (Δ=948.7MB), VRAM peak=12814.9MB

=== Replicate 5/11 ===


...

Rep 7: MSE=0.7319, Time=10.76s, RAM before=0.0MB, peak=948.6MB (Δ=948.6MB), VRAM peak=12814.9MB

=== Replicate 8/11 ===


Rep 8: MSE=0.7419, Time=10.70s, RAM before=0.0MB, peak=948.6MB (Δ=948.6MB), VRAM peak=12814.9MB

=== Replicate 9/11 ===


Rep 9: MSE=0.7412, Time=10.78s, RAM before=0.0MB, peak=948.6MB (Δ=948.6MB), VRAM peak=12814.9MB

=== Replicate 10/11 ===


Rep 10: MSE=0.7401, Time=10.86s, RAM before=0.0MB, peak=948.6MB (Δ=948.6MB), VRAM peak=12814.9MB

=== Replicate 11/11 ===


Rep 11: MSE=0.7412, Time=10.78s, RAM before=0.0MB, peak=948.6MB (Δ=948.6MB), VRAM peak=12814.9MB


In [ ]:
print(statistics.mean(mse_l_rff[1:]))
print(statistics.stdev(mse_l_rff[1:]))

print(statistics.mean(time_l_rff[1:]))
print(statistics.stdev(time_l_rff[1:]))


0.736411714553833
0.005467466117550878
10.717625713348388
0.08704799029705518


In [ ]:
import statistics
import numpy as np

def remove_outliers_iqr(data, k=1.5):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower = q1 - k * iqr
    upper = q3 + k * iqr
    return [x for x in data if lower <= x <= upper]

# Exclude warm-up replicate (index 0) and outliers
mse_filtered = remove_outliers_iqr(mse_l_rff[1:])
time_filtered = remove_outliers_iqr(time_l_rff[1:])

print("MSE (filtered):", statistics.mean(mse_filtered))
print("Std  (MSE):    ", statistics.stdev(mse_filtered))
print("Time (filtered):", statistics.mean(time_filtered))
print("Std  (Time):    ", statistics.stdev(time_filtered))

MSE (filtered): 0.736411714553833
Std  (MSE):     0.005467466117550878
Time (filtered): 10.717625713348388
Std  (Time):     0.08704799029705518
